In [1]:
import time
from collections import defaultdict
from gorgo.core import Bernoulli
from gorgo.inference import SamplePrior, Enumeration

def geometric(p):
    x = Bernoulli(p).sample()
    if x == 0:
        return 0
    return 1 + geometric(p)

## Direct comparison with webppl

In [2]:
runs = 10
start = time.time()
for _ in range(runs):
    dist = Enumeration(geometric, max_executions=100).run(.9)
tot_time = time.time() - start
print(f"Runs: {runs}; Total Time: {tot_time:.1f}s; {tot_time/runs:.5f}s per run")

Runs: 10; Total Time: 0.3s; 0.02660s per run


WebPPL comparison:

```javascript
var geometric = function(p) {
  var x = flip(p)
  if (x == 0) {
    return 0
  }
  return x + geometric(p)
}
var runs = 10
var res = map( function () { return Infer({
  method: 'enumerate',
  maxExecutions: 100,
  model: function () {geometric(.9)}
})}, _.range(runs))
display(res[0])
```

Run details:
```
webppl: v0.9.15-430b433d
packages:
seed: 1670200195749
compile: 0.652s
run: 0.047s
```

## Comparison of CPS interpretation with normal python

In [3]:
from gorgo.interpreter import CPSInterpreter
from gorgo.core import ReturnMessage, ProgramState

_cps = CPSInterpreter()
def geometric_cps(p, *, _cps=_cps, _cont=lambda val: val):
    def _cont_1(_res_1):
        __body_0__body_0__value__func__value = _res_1

        def _cont_2(_res_2):
            x = _res_2
            if x == 0:
                return lambda : _cont(0)
            else:

                def _cont_5(_res_5):
                    __body_0__body_2__value__right = _res_5
                    return lambda : _cont(1 + __body_0__body_2__value__right)
                return lambda : geometric_cps(p, _cont=_cont_5)
        return lambda : _cont_2(__body_0__body_0__value__func__value.sample())
    return lambda : _cps.interpret(Bernoulli)(p, _cont=_cont_1)

In [4]:
start = time.time()
cps_results = defaultdict(int)
n = 10000
for _ in range(n):
    thunk = geometric_cps(.9)
    while callable(thunk):
        thunk = thunk()
    cps_results[thunk] += 1
tot_time = time.time() - start
cps_results = {r: c/n for r, c in cps_results.items()}
cps_exp = sum(r*p for r, p in cps_results.items())
print(f"cps_exp: {cps_exp:.2f}, time: {tot_time:.2f}")

cps_exp: 8.97, time: 0.46


In [5]:
start = time.time()
results = defaultdict(int)
n = 10000
for _ in range(n):
    res = geometric(.9)
    results[res] += 1
tot_time = time.time() - start
results = {r: c/n for r, c in results.items()}
exp = sum(r*p for r, p in results.items())
print(f"exp: {exp:.2f}, time: {tot_time:.2f}")

exp: 9.02, time: 0.11
